# building the classification model 

In [1]:
KEY = 'a9873e8d54214dadb098099712889729'

# getting stared 

In [2]:
!export AZURE_SEARCH_KEY='a9873e8d54214dadb098099712889729'

In [5]:
!pip install fastbook 

     |████████████████████████████████| 727kB 11.7MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 194kB 24.4MB/s 
     |████████████████████████████████| 1.2MB 35.9MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
from fastbook import * 
from fastai.vision.all import * 

In [ ]:
search_images_bing

In [ ]:
results = search_images_bing(KEY,'grizzly bear')

In [ ]:
ims = results.attrgot('contentUrl')

In [ ]:
len(ims),ims[2]

In [11]:
dest = '/content/images/grizzly.jpg'
download_url(ims[0],dest)

im = Image.open(dest)

im.to_thumb(128,128)

FileNotFoundError: ignored

In [12]:
bear_types = ['grizzly','black','teddy']
path =Path('bears')

In [ ]:
if not path.exists():
  path.mkdir()
  for o in bear_types:
    dest = (path/o)
    dest.mkdir(exist_ok = True)
    results = search_images_bing(KEY,f'{o} bear')
    download_images(urls = results.attrgot('contentUrl'),dest = dest)

In [ ]:
fns = get_image_files(path)

In [ ]:
fns

In [ ]:
failed = verify_images(fns)

In [ ]:
failed.map(Path.unlink)

In [ ]:
bear = DataBlock(
    blocks = (ImageBlock,CategoryBlock), # 
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct = 0.2 , seed = 42),
    get_y = parent_label,
    item_tfms = Resize(128)
)

In [ ]:
dls = bear.dataloaders(path)

In [ ]:
dls.show_batch()

In [ ]:
learner = cnn_learner(dls, resnet34,metrics = accuracy).to_fp16()

In [ ]:
learner.fit_one_cycle(3)

In [ ]:
learner.plot_top_losses(5,nrows = 1 ) 

In [ ]:
cleaner = ImageClassifierCleaner(learner)
cleaner